In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter('ignore')
import math
from sklearn.pipeline import Pipeline
from sklearn import set_config
from matplotlib import style
from matplotlib.patches import Ellipse
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler 

In [2]:
import models
hankel = models.Hankel()
rank = models.Rank()
cluster = models.Cluster()
meepc = models.MEEPC() 

In [3]:
df = pd.read_csv('~/data/training_data.csv')

In [4]:
df

,1_AIT_001_PV,1_AIT_002_PV,1_AIT_003_PV,1_AIT_004_PV,1_AIT_005_PV,1_FIT_001_PV,1_LT_001_PV,1_MV_001_STATUS,1_MV_002_STATUS,1_MV_003_STATUS,...,3_AIT_001_PV,3_AIT_002_PV,3_AIT_003_PV,3_AIT_004_PV,3_AIT_005_PV,3_FIT_001_PV,3_LT_001_PV,LEAK_DIFF_PRESSURE,TOTAL_CONS_REQUIRED_FLOW,"Attack LABLE (1:No Attack, -1:Attack)"
0,167.892,0.625471,11.6175,504.140,0.293168,0.001107,62.1436,1.0,1.0,1.0,...,0.0,8279.1,14.35530,238.111,0.620147,0.667695,67.0974,60.0435,0.11,1.0
1,167.892,0.625471,11.6175,504.140,0.293168,0.001107,62.1436,1.0,1.0,1.0,...,0.0,8279.1,14.35530,238.111,0.620147,0.667695,67.0974,60.0435,0.11,1.0
2,167.892,0.625471,11.6175,504.140,0.293168,0.001107,62.1436,1.0,1.0,1.0,...,0.0,8279.1,14.35530,238.111,0.620147,0.667695,67.0974,60.0435,0.11,1.0
3,167.892,0.625471,11.6175,504.140,0.293168,0.001107,62.1436,1.0,1.0,1.0,...,0.0,8279.1,14.35530,238.111,0.620147,0.667695,67.0974,60.0435,0.11,1.0
4,167.895,0.619473,11.6179,504.145,0.293009,0.001080,62.1472,1.0,1.0,1.0,...,0.0,8279.1,14.00090,264.802,0.620240,0.672394,67.1004,60.1557,0.11,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971995,179.618,0.625473,11.9156,420.420,0.228225,1.903020,43.3472,2.0,1.0,1.0,...,0.0,8279.1,5.21471,783.722,0.536200,0.627771,64.5326,86.5871,1.47,-1.0
971996,179.618,0.625473,11.9156,420.420,0.228225,1.903020,43.3472,2.0,1.0,1.0,...,0.0,8279.1,5.21471,783.722,0.536200,0.627771,64.5326,86.5871,1.47,-1.0
971997,179.618,0.625473,11.9156,420.420,0.228225,1.903020,43.3472,2.0,1.0,1.0,...,0.0,8279.1,5.21471,783.722,0.536200,0.627771,64.5326,86.5871,1.47,-1.0
971998,179.618,0.625473,11.9156,420.420,0.228225,1.903020,43.3472,2.0,1.0,1.0,...,0.0,8279.1,5.21471,783.722,0.536200,0.627771,64.5326,86.5871,1.47,-1.0


In [5]:
df_normal=df[df['Attack LABLE (1:No Attack, -1:Attack)']==1]
df_attack=df[df['Attack LABLE (1:No Attack, -1:Attack)']==-1]

In [6]:
df_normal

,1_AIT_001_PV,1_AIT_002_PV,1_AIT_003_PV,1_AIT_004_PV,1_AIT_005_PV,1_FIT_001_PV,1_LT_001_PV,1_MV_001_STATUS,1_MV_002_STATUS,1_MV_003_STATUS,...,3_AIT_001_PV,3_AIT_002_PV,3_AIT_003_PV,3_AIT_004_PV,3_AIT_005_PV,3_FIT_001_PV,3_LT_001_PV,LEAK_DIFF_PRESSURE,TOTAL_CONS_REQUIRED_FLOW,"Attack LABLE (1:No Attack, -1:Attack)"
0,167.892,0.625471,11.6175,504.140,0.293168,0.001107,62.1436,1.0,1.0,1.0,...,0.0,8279.1,14.35530,238.111,0.620147,0.667695,67.0974,60.0435,0.11,1.0
1,167.892,0.625471,11.6175,504.140,0.293168,0.001107,62.1436,1.0,1.0,1.0,...,0.0,8279.1,14.35530,238.111,0.620147,0.667695,67.0974,60.0435,0.11,1.0
2,167.892,0.625471,11.6175,504.140,0.293168,0.001107,62.1436,1.0,1.0,1.0,...,0.0,8279.1,14.35530,238.111,0.620147,0.667695,67.0974,60.0435,0.11,1.0
3,167.892,0.625471,11.6175,504.140,0.293168,0.001107,62.1436,1.0,1.0,1.0,...,0.0,8279.1,14.35530,238.111,0.620147,0.667695,67.0974,60.0435,0.11,1.0
4,167.895,0.619473,11.6179,504.145,0.293009,0.001080,62.1472,1.0,1.0,1.0,...,0.0,8279.1,14.00090,264.802,0.620240,0.672394,67.1004,60.1557,0.11,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971938,179.491,0.631472,11.9234,420.706,0.228469,1.899090,42.5462,2.0,1.0,1.0,...,0.0,8279.1,7.05323,690.042,0.516764,0.001354,64.3759,68.7421,1.56,1.0
971939,179.491,0.631472,11.9234,420.706,0.228469,1.899090,42.5462,2.0,1.0,1.0,...,0.0,8279.1,7.05323,690.042,0.516764,0.001354,64.3759,68.7421,1.56,1.0
971940,179.491,0.631472,11.9234,420.706,0.228469,1.899090,42.5462,2.0,1.0,1.0,...,0.0,8279.1,7.05323,690.042,0.516764,0.001354,64.3759,68.7421,1.56,1.0
971941,179.491,0.631472,11.9234,420.706,0.228469,1.899090,42.5462,2.0,1.0,1.0,...,0.0,8279.1,7.05323,690.042,0.516764,0.001354,64.3759,68.7421,1.56,1.0


In [7]:
df_attack

,1_AIT_001_PV,1_AIT_002_PV,1_AIT_003_PV,1_AIT_004_PV,1_AIT_005_PV,1_FIT_001_PV,1_LT_001_PV,1_MV_001_STATUS,1_MV_002_STATUS,1_MV_003_STATUS,...,3_AIT_001_PV,3_AIT_002_PV,3_AIT_003_PV,3_AIT_004_PV,3_AIT_005_PV,3_FIT_001_PV,3_LT_001_PV,LEAK_DIFF_PRESSURE,TOTAL_CONS_REQUIRED_FLOW,"Attack LABLE (1:No Attack, -1:Attack)"
941103,167.005,0.613474,11.9575,467.477,0.293415,0.000870,44.9750,1.0,1.0,1.0,...,0.0,8279.1,7.33387,633.337,0.491560,0.001369,64.4316,63.2852,0.69,-1.0
941104,167.005,0.613474,11.9575,467.477,0.293415,0.000870,44.9750,1.0,1.0,1.0,...,0.0,8279.1,7.33387,633.337,0.491560,0.001369,64.4316,63.2852,0.69,-1.0
941105,167.005,0.613474,11.9575,467.477,0.293415,0.000870,44.9750,1.0,1.0,1.0,...,0.0,8279.1,7.33387,633.337,0.491560,0.001369,64.4316,63.2852,0.69,-1.0
941106,166.965,0.619474,11.9573,467.522,0.293185,0.001254,45.1677,1.0,1.0,1.0,...,0.0,8279.1,7.26840,638.484,0.491449,0.001392,64.3614,63.5099,0.69,-1.0
941107,166.965,0.619474,11.9573,467.522,0.293185,0.001254,45.1677,1.0,1.0,1.0,...,0.0,8279.1,7.26840,638.484,0.491449,0.001392,64.3614,63.5099,0.69,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971995,179.618,0.625473,11.9156,420.420,0.228225,1.903020,43.3472,2.0,1.0,1.0,...,0.0,8279.1,5.21471,783.722,0.536200,0.627771,64.5326,86.5871,1.47,-1.0
971996,179.618,0.625473,11.9156,420.420,0.228225,1.903020,43.3472,2.0,1.0,1.0,...,0.0,8279.1,5.21471,783.722,0.536200,0.627771,64.5326,86.5871,1.47,-1.0
971997,179.618,0.625473,11.9156,420.420,0.228225,1.903020,43.3472,2.0,1.0,1.0,...,0.0,8279.1,5.21471,783.722,0.536200,0.627771,64.5326,86.5871,1.47,-1.0
971998,179.618,0.625473,11.9156,420.420,0.228225,1.903020,43.3472,2.0,1.0,1.0,...,0.0,8279.1,5.21471,783.722,0.536200,0.627771,64.5326,86.5871,1.47,-1.0


In [8]:
sensors = [3,9,25,43]
df_normal = df_normal.iloc[:, sensors]
df_attack=df_attack.iloc[:,sensors]

In [9]:
def calc_width_height(v):
    width = 2/np.sqrt(v) 
    return width
    
def draw_ellipse(i,j,centers,widths,X):
    plt.xlim(centers[i]-widths[i]/2,centers[i]+widths[i]/2)
    plt.ylim(centers[j]-widths[j]/2,centers[j]+widths[j]/2)
    ellipse = Ellipse(xy=(centers[i], centers[j]), width=widths[i], height=widths[j])
    ellipse.fill = False
    ellipse.set_ec('red')
    ax.add_patch(ellipse)
    plt.scatter(x = X[::,i], y = X[::,j])
    

In [10]:
X = df_normal.iloc[:,2].values
X = hankel.fit(X,30,0.5)
X=X.T # size--> 1069X1800
kmeans,optimal_k = cluster.fit(X)
print(optimal_k)
radiis = []
weights=[]
centers=[]
clusters_R=[]
clusters_V=[]
# tsne=TSNE(n_components=2,perplexity=30)

for i in range(optimal_k):
    cluster_ = X[np.where(kmeans.labels_ == i)[0]]
    r = rank.fit(cluster_)
    clusters_R.append(r)
    print("r: "+str(r))
    U,Sigma,VT = np.linalg.svd(cluster_)
    V = VT.T
    clusters_V.append(V)
    cluster_ = np.matmul(cluster_,V[:,:r])
    # cluster_=tsne.fit_transform(cluster_)
    weight,center = meepc.fit(cluster_)
    weights.append(weight)
    centers.append(center)
    widths=calc_width_height(weight)
    var1=np.square(cluster_-center)
    var2=np.matmul(weight,var1.T)
    max_radius=math.sqrt(np.amax(var2))
    radiis.append(max_radius)
    #plotting
#     ax = plt.gca()
#     ellipse = Ellipse(xy=(center[0],center[1]), width=widths[0], height=widths[1])
#     ellipse.fill = False 
#     ellipse.set_ec('red')
#     ax.add_patch(ellipse)
#     sns.scatterplot(x = cluster_[:,0], y = cluster_[:,1])
# plt.show()
print(len(radiis))

4
r: 14
r: 13
r: 12
r: 12
4


# attack training


In [11]:
from sklearn.cluster import KMeans

In [12]:
X_attack = df_attack.iloc[:,2].values
X_att = hankel.fit(X_attack,30,0.5)
X_att=X_att.T # size--> X1800
kmeans = KMeans(n_clusters=optimal_k, init='k-means++')
kmeans.fit(X_att)
radiis2=[]
print(X_att.shape)
for i in range(optimal_k):
    cluster_ = X_att[np.where(kmeans.labels_ == i)[0]]
    r = rank.fit(cluster_)
    # clusters_R.append(r)
    print("r: "+str(r))
    U,Sigma,VT = np.linalg.svd(cluster_)
    V = VT.T
    clusters_V.append(V)
    cluster_ = np.matmul(cluster_,V[:,:r])
    # cluster_=tsne.fit_transform(cluster_)
    weight,center = meepc.fit(cluster_)
    weights.append(weight)
    centers.append(center)
    widths=calc_width_height(weight)
    var1=np.square(cluster_-center)
    var2=np.matmul(weight,var1.T)
    # max_radius=math.sqrt(np.amax(var2))
    radiis2.append(var2)
print(len(radiis))

(8, 1800)
r: 1
r: 2
r: 0
r: 1
4


In [13]:
radiis

[array([0.27895858, 0.27895858, 0.27895858, 0.27895858, 0.27895858,
        0.27895858, 0.27895659, 0.27895739, 0.27895858, 0.27895858,
        0.27895858, 0.27895858, 0.27895858, 0.27895858, 0.27895858,
        0.66342504, 0.27895858, 0.27895858, 0.27894069, 0.27895858,
        0.27895858, 0.27895858, 0.27880066, 0.27895858, 0.27895858,
        0.27895858, 0.27895858, 0.26223364, 0.27895858, 0.27895858,
        0.27500693, 0.30335344, 0.10905187, 0.27895858, 0.27895858,
        0.90116302, 0.2638996 , 0.27895858, 0.27895858, 0.27895858,
        0.27895858, 0.27895858, 0.27895858, 0.27603768, 0.32767687,
        0.27895858, 0.27895858, 0.27895858, 0.27895858, 0.27895858,
        0.43706273, 0.27895858, 0.27895858, 0.27895858, 0.27895858,
        0.27895858, 0.27895858, 0.27895858, 0.27895858, 0.27895858,
        0.27895858, 0.27895858, 0.55943853, 0.46423457, 0.27895858,
        0.64556662, 0.45923902, 0.26498464, 0.27895858, 0.27895858,
        0.27895858, 0.27895858, 0.27895858, 0.32

In [14]:
radiis2

[array([1., 1.]),
 array([0.99999596, 1.00000553, 1.73566191]),
 array([0.]),
 array([1., 1.])]

In [15]:
# if math.isnan(max_radius):
#         print('Rank: ',r)
#         print('Cluster Shape: ',cluster.shape)
#         print('CLuster: ',cluster)
#         print('Weight Shape: ', weight.shape)
#         print('Weight: ', weight)
#         print('Center Shape: ', center.shape)
#         print('Center: ', center)
#         print('Var1: ', var1)
#         print('Var2: ', var2)

# testing 

In [16]:
df_test=pd.read_csv('~/data/testdata.csv')
df_test

,1_AIT_001_PV,1_AIT_002_PV,1_AIT_003_PV,1_AIT_004_PV,1_AIT_005_PV,1_FIT_001_PV,1_LT_001_PV,1_MV_001_STATUS,1_MV_002_STATUS,1_MV_003_STATUS,...,3_AIT_001_PV,3_AIT_002_PV,3_AIT_003_PV,3_AIT_004_PV,3_AIT_005_PV,3_FIT_001_PV,3_LT_001_PV,LEAK_DIFF_PRESSURE,TOTAL_CONS_REQUIRED_FLOW,"Attack LABLE (1:No Attack, -1:Attack)"
0,161.129,0.721464,11.7842,463.716,0.267211,1.890010,48.9329,2.0,1.0,1.0,...,0.0,8279.1,5.95083,743.356,0.594601,0.001333,64.5137,74.2107,1.92,1.0
1,161.129,0.721464,11.7842,463.716,0.267211,1.890010,48.9329,2.0,1.0,1.0,...,0.0,8279.1,5.95083,743.356,0.594601,0.001333,64.5137,74.2107,1.92,1.0
2,161.129,0.721464,11.7842,463.716,0.267211,1.890010,48.9329,2.0,1.0,1.0,...,0.0,8279.1,5.95083,743.356,0.594601,0.001333,64.5137,74.2107,1.92,1.0
3,161.129,0.721464,11.7842,463.716,0.267211,1.890010,48.9329,2.0,1.0,1.0,...,0.0,8279.1,5.95083,743.356,0.594601,0.001333,64.5137,74.2107,1.92,1.0
4,161.129,0.721464,11.7842,463.716,0.267211,1.890010,48.9329,2.0,1.0,1.0,...,0.0,8279.1,5.95083,743.356,0.594601,0.001333,64.5137,74.2107,1.92,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248395,176.467,0.613474,11.9281,430.680,0.228327,0.001214,66.1851,1.0,1.0,1.0,...,0.0,8279.1,13.83640,370.386,0.729505,0.595032,64.3879,66.1173,1.41,1.0
248396,176.467,0.613474,11.9281,430.680,0.228327,0.001214,66.1851,1.0,1.0,1.0,...,0.0,8279.1,13.83640,370.386,0.729505,0.595032,64.3879,66.1173,1.41,1.0
248397,176.467,0.613474,11.9281,430.680,0.228327,0.001214,66.1851,1.0,1.0,1.0,...,0.0,8279.1,13.83640,370.386,0.729505,0.595032,64.3879,66.1173,1.41,1.0
248398,176.467,0.613474,11.9281,430.680,0.228327,0.001214,66.1851,1.0,1.0,1.0,...,0.0,8279.1,13.83640,370.386,0.729505,0.595032,64.3879,66.1173,1.41,1.0


In [17]:
y_actual=df_test.iloc[:,-1]
# df_test_normal=df_test[df_test['Attack LABLE (1:No Attack, -1:Attack)']==1]
sensors = [3,9,25,4]
df_test=df_test.iloc[:,sensors]
df_test


,1_AIT_004_PV,1_MV_003_STATUS,2_FIC_401_CO,1_AIT_005_PV
0,463.716,1.0,20.2507,0.267211
1,463.716,1.0,20.2507,0.267211
2,463.716,1.0,20.2507,0.267211
3,463.716,1.0,20.2507,0.267211
4,463.716,1.0,20.2507,0.267211
...,...,...,...,...
248395,430.680,1.0,19.2170,0.228327
248396,430.680,1.0,19.2170,0.228327
248397,430.680,1.0,19.2170,0.228327
248398,430.680,1.0,19.2170,0.228327


In [28]:
labels=hankel.fit(np.array(y_actual),30,0.5)
labels

array([[ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.],
       ...,
       [ 1.,  1.,  1., ...,  1., -1.,  1.],
       [ 1.,  1.,  1., ...,  1., -1.,  1.],
       [ 1.,  1.,  1., ...,  1., -1.,  1.]])

In [34]:
isatt=np.any(labels<0,axis=0)
att_indices=np.where(isatt)[0]
att_indices

array([267, 268, 269, 272, 273])

In [19]:
X_test=df_test.iloc[:,2].values
Xtest=hankel.fit(X_test,30,0.5)  #size -> 1800x274
Xtest=Xtest.T  #size -> 274x1800
print(Xtest.shape)
distances=[]
for i in range(optimal_k):
    Xtest_cluster=np.matmul(Xtest,clusters_V[i][:,:clusters_R[i]]) #size -> 274 x R
    var1=np.square(Xtest_cluster-centers[i])
    var2=np.matmul(weights[i],var1.T)
    distance=np.sqrt(var2)
    distances.append(distance)

(274, 1800)


In [20]:
distances[0].shape #size (274,)

(274,)

In [21]:
# distances matrix
stacked_arr=np.vstack(distances)
distances_arr=np.transpose(stacked_arr)
distances_arr #-> 274 X 4

array([[1.91183702, 1.23307772, 0.82753116, 1.88980583],
       [0.59516904, 1.45744475, 1.30231661, 1.52712792],
       [0.5280319 , 1.44444121, 1.18354432, 1.38266788],
       ...,
       [1.22403106, 1.01382955, 1.33924728, 0.40179398],
       [1.85588754, 0.42158724, 0.89789537, 1.09004837],
       [1.93499066, 0.50912986, 0.88750514, 1.08380835]])

In [22]:
radiis

[array([0.27895858, 0.27895858, 0.27895858, 0.27895858, 0.27895858,
        0.27895858, 0.27895659, 0.27895739, 0.27895858, 0.27895858,
        0.27895858, 0.27895858, 0.27895858, 0.27895858, 0.27895858,
        0.66342504, 0.27895858, 0.27895858, 0.27894069, 0.27895858,
        0.27895858, 0.27895858, 0.27880066, 0.27895858, 0.27895858,
        0.27895858, 0.27895858, 0.26223364, 0.27895858, 0.27895858,
        0.27500693, 0.30335344, 0.10905187, 0.27895858, 0.27895858,
        0.90116302, 0.2638996 , 0.27895858, 0.27895858, 0.27895858,
        0.27895858, 0.27895858, 0.27895858, 0.27603768, 0.32767687,
        0.27895858, 0.27895858, 0.27895858, 0.27895858, 0.27895858,
        0.43706273, 0.27895858, 0.27895858, 0.27895858, 0.27895858,
        0.27895858, 0.27895858, 0.27895858, 0.27895858, 0.27895858,
        0.27895858, 0.27895858, 0.55943853, 0.46423457, 0.27895858,
        0.64556662, 0.45923902, 0.26498464, 0.27895858, 0.27895858,
        0.27895858, 0.27895858, 0.27895858, 0.32

In [23]:
check_anamoly=radiis-distances_arr     # positive = inside cluster | negative = outside cluster
check_anamoly

array([[array([-1.63287843, -1.63287843, -1.63287843, -1.63287843, -1.63287843,
               -1.63287843, -1.63288042, -1.63287963, -1.63287843, -1.63287843,
               -1.63287843, -1.63287843, -1.63287843, -1.63287843, -1.63287843,
               -1.24841198, -1.63287843, -1.63287843, -1.63289633, -1.63287843,
               -1.63287843, -1.63287843, -1.63303636, -1.63287843, -1.63287843,
               -1.63287843, -1.63287843, -1.64960338, -1.63287843, -1.63287843,
               -1.63683009, -1.60848357, -1.80278514, -1.63287843, -1.63287843,
               -1.010674  , -1.64793741, -1.63287843, -1.63287843, -1.63287843,
               -1.63287843, -1.63287843, -1.63287843, -1.63579934, -1.58416014,
               -1.63287843, -1.63287843, -1.63287843, -1.63287843, -1.63287843,
               -1.47477429, -1.63287843, -1.63287843, -1.63287843, -1.63287843,
               -1.63287843, -1.63287843, -1.63287843, -1.63287843, -1.63287843,
               -1.63287843, -1.63287843,

In [30]:
check_anamoly.shape

(274, 4)

In [35]:
#check for if any datapoint is coming inside the cluster
is_positive=np.any(check_anamoly>0,axis=1)
#indices of those rows
positive_indices=np.where(is_positive)[0]
print(positive_indices)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
len(positive_indices)

269